In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from __future__ import print_function
from __future__ import division

import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
 
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #有多个GPU时可以指定只使用第几号GPU
config = tf.compat.v1.ConfigProto()
config.allow_soft_placement=True #允许动态放置张量和操作符
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth=True   #初始化时不全部占满GPU显存, 按需分配 
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
import os
import csv
from PIL import Image
from cv2 import cv2
import random

In [0]:
# categorical_crossentropy
#def load_data(): 
x_train = []
y_train = []
x_test = []
y_test = []
size = (168,126)
i = 1
#with open(r'/content/drive/My Drive/C1-P1_Train Dev_fixed/train.csv', newline='') as csvFile:
#with open(r'/content/drive/My Drive/C1-P1_Train Dev_fixed_s/train.csv', newline='') as csvFile:
with open(r'/content/drive/My Drive/C1-P1_Train/train.csv', newline='') as csvFile:
    rows = csv.reader(csvFile)
    rows = csv.reader(csvFile, delimiter=',')
    for row in rows:
        if(row[1]!='label'):
            if(row[1]=='A'):
                label = 0
            elif(row[1]=='B'):
                label = 1
            elif(row[1]=='C'):
                label = 2
            #img = Image.open(os.path.join('/content/drive/My Drive/C1-P1_Train Dev_fixed/C1-P1_Train', row[0]))
            img = Image.open(os.path.join('/content/drive/My Drive/C1-P1_Train', row[0]))
            img = img.resize(size,Image.BILINEAR)
            imgarray = np.array(img)
            imgarray = (imgarray - np.min(imgarray))/(np.max(imgarray)-np.min(imgarray))
            x_train.append(imgarray)
            y_train.append(label)
            print(i,row[0],row[1])
            i = i + 1
j = 1
with open(r'/content/drive/My Drive/C1-P1_Train Dev_fixed/dev.csv', newline='') as csvFile:
#with open(r'/content/drive/My Drive/C1-P1_Train Dev_fixed_s/dev.csv', newline='') as csvFile:
    rows = csv.reader(csvFile)
    rows = csv.reader(csvFile, delimiter=',')
    for row in rows:
        if(row[1]!='label'):
            if(row[1]=='A'):
                label = 0
            elif(row[1]=='B'):
                label = 1
            elif(row[1]=='C'):
                label = 2
            img = Image.open(os.path.join('/content/drive/My Drive/C1-P1_Train Dev_fixed/C1-P1_Dev', row[0]))
            img = img.resize(size,Image.BILINEAR)
            imgarray = np.array(img)
            imgarray = (imgarray - np.min(imgarray))/(np.max(imgarray)-np.min(imgarray))
            x_test.append(imgarray)
            y_test.append(label)
            print(j,row[0],row[1])
            j = j + 1
            
# convert class vectors to binary class matrices
temp = list(zip(x_train, y_train))
random.shuffle(temp)
x_train, y_train = zip(*temp)
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
y_train = np_utils.to_categorical(y_train, 3)
y_test = np_utils.to_categorical(y_test, 3)

print('data loaded')

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
train_data_gen = datagen.flow(x_train, y_train, batch_size=100)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop
from keras.callbacks import EarlyStopping
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.engine import training
from typing import Tuple
import time
EStop = EarlyStopping(monitor='val_accuracy', min_delta=0, 
                      patience=10, verbose=1, mode='auto')
optimizer = RMSprop(lr=1e-4)

"\nmodel = Sequential()\nmodel.add(Conv2D(64, (3, 3), activation='relu', input_shape=(126,168,3),padding='same',name='block1_conv2_1'))\nmodel.add(Conv2D(64, (3, 3), activation='relu',padding='same',name='block1_conv2_2'))\nmodel.add(MaxPooling2D(pool_size=(2, 2),name='block1_MaxPooling'))\nmodel.add(SpatialDropout2D(0.25))\nmodel.add(Conv2D(128, (3, 3), activation='relu',padding='same',name='block2_conv2_1'))\nmodel.add(Conv2D(128, (3, 3), activation='relu',padding='same',name='block2_conv2_2'))\nmodel.add(MaxPooling2D(pool_size=(2, 2),name='block2_MaxPooling'))\nmodel.add(SpatialDropout2D(0.25))\nmodel.add(Flatten())\nmodel.add(Dense(128, activation='relu',name='final_output_1'))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(128, activation='relu',name='final_output_2'))\nmodel.add(Dropout(0.5))\nmodel.add(Dense(3, activation='sigmoid',name='class_output'))\noptimizer = RMSprop(lr=1e-4)\nobjective = 'binary_crossentropy'\nmodel.compile(loss=objective, optimizer=optimizer, metrics=['accur

In [0]:
###
#DenseNet
###
from keras.applications.densenet import DenseNet121

num_classes = 3
input_shape = (126, 168, 3)
model_densenet = DenseNet121(include_top=True, weights=None,
                  input_shape=input_shape, classes=num_classes)
model_densenet.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model_densenet.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 126, 168, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 132, 174, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 63, 84, 64)   9408        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 63, 84, 64)   256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [0]:
# Training and saving
total_train = 5600
total_val = 800 
history_d = model_densenet.fit_generator(
    train_data_gen,
    steps_per_epoch=64,
    epochs=100,
    callbacks=[EStop],
    validation_data=(x_test, y_test)
)
timestr = time.strftime("%Y%m%d_%H%M%S")
model_densenet.save('model_densenet_{}.h5'.format(timestr))

In [0]:
# Training History
import collections
import pandas as pd

hist_d = history_d.history
for key, val in hist_d.items(): # Count the number of epoch
    numepo = len(np.asarray(val))
    break

import matplotlib.pyplot as plt

plt.plot(history_d.history['accuracy'])
plt.plot(history_d.history['val_accuracy'])
plt.title('DenseNet accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
plt.savefig('Model_d accuracy_{}.png'.format(timestr))
plt.cla()

plt.plot(history_d.history['loss'])
plt.plot(history_d.history['val_loss'])
plt.title('DenseNet loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
plt.savefig('Model_d loss_{}.png'.format(timestr))
plt.close()

In [0]:
import itertools
from sklearn.metrics import confusion_matrix
from keras.utils import plot_model

def plot_confusion_matrix(cm, classes_x,classes_y,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.clf()
    plt.cla()
    plt.style.use('default')
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.grid(False)
    plt.title(title)
    plt.colorbar()
    tick_marks_x = np.arange(len(classes_x))
    tick_marks_y = np.arange(len(classes_y))
    plt.xticks(tick_marks_x, classes_x, rotation=45)
    plt.yticks(tick_marks_y, classes_y)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.savefig(title+'.png',dpi=350 ,bbox_inches='tight')
    plt.show()
    plt.close()

test_pred_d  = model_densenet.predict(x_test)
cnf_matrix_d = confusion_matrix(np.argmax(y_test, axis=1).reshape(-1,1),
                              np.argmax(test_pred_d, axis=1).reshape(-1,1))
np.set_printoptions(precision=2)
plot_confusion_matrix(cnf_matrix_d,['A','B','C'],['A','B','C'],normalize=True,title='DenseNet Confusion Matrix')

Using TensorFlow backend.


NameError: ignored